<a href="https://colab.research.google.com/github/mariabandeira/AdministracaoPublica/blob/main/CadUnico/KNN/TrainningPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [6]:
treated_data_RN = pd.read_csv('./BasesRN/cadunico_rn_cleaned.csv', sep=';', encoding='utf-8')

In [7]:
treated_data_RN.dropna(inplace=True)
treated_data_RN.drop_duplicates(inplace=True)

### Treinando o modelo RNN

In [8]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = treated_data_RN['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution, 2))

marc_pbf
1    56.74
0    43.26
Name: proportion, dtype: float64


In [9]:
# separar X (features) e y (target)
X = treated_data_RN.drop('marc_pbf', axis=1)
y = treated_data_RN['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criando uma instância do RandomUnderSampling
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# balanceando os dados
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(y_resampled.value_counts())

marc_pbf
0    89679
1    89679
Name: count, dtype: int64


In [10]:
# aplicar padronização nas features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print("Primeiros dados escalados (padronizados):")
print(X_train_scaled[:5])

Primeiros dados escalados (padronizados):
[[-0.84487543 -1.96467582 -0.12485031 -0.57790393 -0.602914   -1.16411791
  -0.72537277 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997 -0.85437521  0.05656285 -1.37479098]
 [-0.55264356  0.71176597  0.13504909 -0.57790393 -0.602914    0.30866763
  -0.72537277 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997  1.22999908  0.05656285  0.90668609]
 [-0.75095372 -0.62645493  4.64343735  1.7303914   0.1538476  -1.16411791
   1.28227208 -0.28287978 -0.39178363  1.7490837  -0.16908028  2.00891719
  -0.23932997  1.22999908  0.05656285 -1.37479098]
 [ 0.81400362  0.71176597  1.40335816  1.7303914   3.93765562  1.78145316
   1.28227208 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997  0.18781194  0.05656285  0.14619374]
 [ 1.32648826 -0.62645493 -0.81791538 -0.57790393  0.91060921  0.30866763
   1.28227208 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997 

In [11]:
# Ajustar formato para a RNN (samples, time steps, features)
X_train_rnn = np.expand_dims(X_train_scaled, axis=1)
X_test_rnn = np.expand_dims(X_test_scaled, axis=1)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
# Construir a RNN
model = Sequential([
    SimpleRNN(32, activation='relu', input_shape=(1, X_train.shape[1])),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [17]:
model.fit(X_train_rnn, y_resampled, epochs=20, batch_size=32, validation_data=(X_test_rnn, y_test), callbacks=[early_stop])

Epoch 1/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8520 - loss: 0.3598 - val_accuracy: 0.8889 - val_loss: 0.2996
Epoch 2/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8831 - loss: 0.3064 - val_accuracy: 0.8892 - val_loss: 0.2992
Epoch 3/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.8857 - loss: 0.3014 - val_accuracy: 0.8904 - val_loss: 0.2956
Epoch 4/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8863 - loss: 0.2996 - val_accuracy: 0.8922 - val_loss: 0.2994
Epoch 5/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.8867 - loss: 0.2993 - val_accuracy: 0.8916 - val_loss: 0.2984
Epoch 6/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.8884 - loss: 0.2971 - val_accuracy: 0.8945 - val_loss: 0.2935
Epoch 7/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.8891 - loss: 0.2944 - val_accuracy: 0.8939 - val_loss: 0.2930
Epoch 8/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8878 - loss: 0

In [18]:
y_pred_prob = model.predict(X_test_rnn)
y_pred = (y_pred_prob > 0.5).astype(int)

2776/2776 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.82      0.87     38411
           1       0.88      0.95      0.91     50408

    accuracy                           0.89     88819
   macro avg       0.90      0.89      0.89     88819
weighted avg       0.90      0.89      0.89     88819



### Selecionando features com SelectKBests

In [20]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

Características selecionadas: Index(['vlr_renda_media_fam', 'qtd_comodos_domic_fam', 'cod_material_piso_fam',
       'cod_calcamento_domic_fam', 'qtde_pessoas'],
      dtype='object')


In [21]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

In [22]:
X_train_rnn = np.expand_dims(X_train_scaled, axis=1)
X_test_rnn = np.expand_dims(X_test_scaled, axis=1)

In [23]:
model = Sequential([
    SimpleRNN(32, activation='relu', input_shape=(1, X_train_scaled.shape[1])),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [26]:
model.fit(X_train_rnn, y_resampled, epochs=20, batch_size=32, validation_data=(X_test_rnn, y_test), callbacks=[early_stop])

Epoch 1/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.8620 - loss: 0.3406 - val_accuracy: 0.8884 - val_loss: 0.3008
Epoch 2/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8855 - loss: 0.3035 - val_accuracy: 0.8931 - val_loss: 0.2959
Epoch 3/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.8875 - loss: 0.3002 - val_accuracy: 0.8934 - val_loss: 0.2959
Epoch 4/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8871 - loss: 0.3018 - val_accuracy: 0.8927 - val_loss: 0.2988
Epoch 5/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8874 - loss: 0.3008 - val_accuracy: 0.8935 - val_loss: 0.2964
Epoch 6/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8880 - loss: 0.2992 - val_accuracy: 0.8940 - val_loss: 0.2952
Epoch 7/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8891 - loss: 0.2975 - val_accuracy: 0.8948 - val_loss: 0.2926
Epoch 8/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8881 - loss: 0

In [27]:
y_pred_prob = model.predict(X_test_rnn)
y_pred = (y_pred_prob > 0.5).astype(int)

2776/2776 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.82      0.87     38411
           1       0.88      0.95      0.91     50408

    accuracy                           0.90     88819
   macro avg       0.90      0.89      0.89     88819
weighted avg       0.90      0.90      0.90     88819

